Query 1: Executive KPI Summary

In [0]:
SELECT 
    COUNT(*) as TotalBookings,
    SUM(IsCompleted) as CompletedBookings,
    ROUND(SUM(IsCompleted) / COUNT(*) * 100, 2) as SuccessRate_Percent,
    SUM(IsCustomerCancelled + IsDriverCancelled) as CancelledBookings,
    ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as CancellationRate_Percent,
    SUM(IsNoDriverFound + IsIncomplete) as IncompleteBookings,
    ROUND(SUM(IsNoDriverFound + IsIncomplete) / COUNT(*) * 100, 2) as IncompleteRate_Percent,
    SUM(IsDriverCancelled) as DriverCancelled,
    SUM(IsCustomerCancelled) as CustomerCancelled
FROM uber_data_clean

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Databricks visualization. Run in Databricks to view.

Monthly Trends

In [0]:
SELECT 
    Month,
    MonthName,
    COUNT(*) as TotalBookings,
    SUM(IsCompleted) as CompletedBookings,
    ROUND(SUM(IsCompleted) / COUNT(*) * 100, 2) as SuccessRate_Percent,
    ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as CancellationRate_Percent,
    ROUND(SUM(BookingValue), 0) as TotalRevenue
FROM uber_data_clean
GROUP BY Month, MonthName
ORDER BY Month

Databricks visualization. Run in Databricks to view.

CANCELLATION BY DAY OF WEEK

In [0]:
SELECT 
    DayOfWeek,
    DayOfWeekName,
    IsWeekend,
    COUNT(*) as TotalBookings,
    SUM(IsCompleted) as CompletedBookings,
    SUM(IsCustomerCancelled + IsDriverCancelled) as CancelledBookings,
    ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as CancellationRate_Percent,
    ROUND(SUM(BookingValue), 0) as TotalRevenue,
    ROUND(AVG(CustomerRating), 2) as AvgCustomerRating
FROM uber_data_clean
GROUP BY DayOfWeek, DayOfWeekName, IsWeekend
ORDER BY DayOfWeek

Databricks visualization. Run in Databricks to view.

CANCELLATION BY HOUR

In [0]:
SELECT 
    Hour,
    COUNT(*) as TotalBookings,
    SUM(IsCompleted) as CompletedBookings,
    SUM(IsCustomerCancelled + IsDriverCancelled) as CancelledBookings,
    SUM(IsCustomerCancelled) as CustomerCancellations,
    SUM(IsDriverCancelled) as DriverCancellations,
    SUM(IsNoDriverFound) as NoDriverFound,
    ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as CancellationRate_Percent,
    ROUND(SUM(BookingValue), 0) as TotalRevenue,
    ROUND(AVG(AvgVTAT), 2) as AvgWaitTime_Min
FROM uber_data_clean
WHERE Hour IS NOT NULL
GROUP BY Hour
ORDER BY Hour

Databricks visualization. Run in Databricks to view.

Wait Time vs Cancellation

In [0]:
SELECT 
    CASE 
        WHEN AvgVTAT <= 2 THEN 'Under 2 min'
        WHEN AvgVTAT <= 5 THEN '2-5 min'
        WHEN AvgVTAT <= 10 THEN '5-10 min'
        WHEN AvgVTAT <= 15 THEN '10-15 min'
        ELSE 'Over 15 min'
    END as WaitTimeBucket,
    CASE 
        WHEN AvgVTAT <= 2 THEN 1
        WHEN AvgVTAT <= 5 THEN 2
        WHEN AvgVTAT <= 10 THEN 3
        WHEN AvgVTAT <= 15 THEN 4
        ELSE 5
    END as SortOrder,
    COUNT(*) as TotalBookings,
    SUM(IsCompleted) as CompletedBookings,
    SUM(IsCustomerCancelled + IsDriverCancelled) as CancelledBookings,
    ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as CancellationRate_Percent,
    ROUND(AVG(BookingValue), 2) as AvgBookingValue
FROM uber_data_clean
WHERE AvgVTAT > 0
GROUP BY 
    CASE 
        WHEN AvgVTAT <= 2 THEN 'Under 2 min'
        WHEN AvgVTAT <= 5 THEN '2-5 min'
        WHEN AvgVTAT <= 10 THEN '5-10 min'
        WHEN AvgVTAT <= 15 THEN '10-15 min'
        ELSE 'Over 15 min'
    END,
    CASE 
        WHEN AvgVTAT <= 2 THEN 1
        WHEN AvgVTAT <= 5 THEN 2
        WHEN AvgVTAT <= 10 THEN 3
        WHEN AvgVTAT <= 15 THEN 4
        ELSE 5
    END
ORDER BY SortOrder

Databricks visualization. Run in Databricks to view.

Customer Cancellation Reasons

In [0]:
SELECT 
    CustomerCancellationReason as Reason,
    COUNT(*) as Count,
    ROUND(COUNT(*) / SUM(COUNT(*)) OVER() * 100, 2) as Percentage
FROM uber_data_clean
WHERE IsCustomerCancelled = 1 AND CustomerCancellationReason != 'Unknown'
GROUP BY CustomerCancellationReason
ORDER BY Count DESC

Databricks visualization. Run in Databricks to view.

Driver Cancellation Reasons


In [0]:
SELECT 
    DriverCancellationReason as Reason,
    COUNT(*) as Count,
    ROUND(COUNT(*) / SUM(COUNT(*)) OVER() * 100, 2) as Percentage
FROM uber_data_clean
WHERE IsDriverCancelled = 1 AND DriverCancellationReason != 'Unknown'
GROUP BY DriverCancellationReason
ORDER BY Count DESC

Databricks visualization. Run in Databricks to view.

Vehicle Type & Time Period Performance

In [0]:
SELECT 
    VehicleType,
    TimePeriod,
    COUNT(*) as TotalBookings,
    SUM(IsCompleted) as CompletedBookings,
    ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as CancellationRate_Percent,
    ROUND(SUM(BookingValue), 0) as Revenue,
    ROUND(AVG(AvgVTAT), 2) as AvgWaitTime_Min
FROM uber_data_clean
WHERE VehicleType IS NOT NULL AND TimePeriod IS NOT NULL
GROUP BY VehicleType, TimePeriod
HAVING COUNT(*) > 20
ORDER BY CancellationRate_Percent DESC

Databricks visualization. Run in Databricks to view.

Incomplete Rides Breakdown

In [0]:
SELECT 
    CASE 
        WHEN IsIncomplete = 1 THEN 'Incomplete'
    END as IncompleteType,
    IncompleteRideReason as Reason,
    COUNT(*) as Count,
    ROUND(COUNT(*) / SUM(COUNT(*)) OVER() * 100, 2) as Percentage
FROM uber_data_clean
WHERE ( IsIncomplete = 1) AND IncompleteRideReason != 'Unknown'
GROUP BY 
    CASE 
        WHEN IsIncomplete = 1 THEN 'Incomplete'
    END,
    IncompleteRideReason
ORDER BY Count DESC

Databricks visualization. Run in Databricks to view.

KEY INSIGHTS


In [0]:
-- Expanded Key Insights Query
-- Finds the highest cancellation across multiple dimensions

SELECT 
    'Most Common Cancellation Type' as InsightType,
    (SELECT CONCAT(
        CASE 
            WHEN SUM(IsCustomerCancelled) > SUM(IsDriverCancelled) THEN 'Customer'
            ELSE 'Driver'
        END
    ) FROM uber_data_clean) as Value

UNION ALL

SELECT 
    'Highest Cancellation Rate Vehicle' as InsightType,
    (SELECT VehicleType FROM (
        SELECT VehicleType, ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as rate
        FROM uber_data_clean
        GROUP BY VehicleType
        ORDER BY rate DESC
        LIMIT 1
    )) as Value

UNION ALL

SELECT 
    'Lowest Cancellation Rate Vehicle' as InsightType,
    (SELECT VehicleType FROM (
        SELECT VehicleType, ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as rate
        FROM uber_data_clean
        GROUP BY VehicleType
        ORDER BY rate ASC
        LIMIT 1
    )) as Value

UNION ALL

SELECT 
    'Peak Cancellation Hour' as InsightType,
    (SELECT CONCAT(Hour, ':00') FROM (
        SELECT Hour, ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as rate
        FROM uber_data_clean
        WHERE Hour IS NOT NULL
        GROUP BY Hour
        ORDER BY rate DESC
        LIMIT 1
    )) as Value

-- NEW ADDITIONS BELOW

UNION ALL

SELECT 
    'Highest Cancellation Rate Month' as InsightType,
    (SELECT MonthName FROM (
        SELECT Month, MonthName, ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as rate
        FROM uber_data_clean
        GROUP BY Month, MonthName
        ORDER BY rate DESC
        LIMIT 1
    )) as Value

UNION ALL

SELECT 
    'Highest Cancellation Rate Day of Week' as InsightType,
    (SELECT DayOfWeekName FROM (
        SELECT DayOfWeek, DayOfWeekName, ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as rate
        FROM uber_data_clean
        GROUP BY DayOfWeek, DayOfWeekName
        ORDER BY rate DESC
        LIMIT 1
    )) as Value

UNION ALL

SELECT 
    'Highest Cancellation Rate Wait Time Bucket' as InsightType,
    (SELECT WaitBucket
    FROM (
        SELECT 
            CASE 
                WHEN AvgVTAT <= 2 THEN 'Under 2 min'
                WHEN AvgVTAT <= 5 THEN '2-5 min'
                WHEN AvgVTAT <= 10 THEN '5-10 min'
                WHEN AvgVTAT <= 15 THEN '10-15 min'
                ELSE 'Over 15 min'
            END as WaitBucket,
            ROUND(SUM(IsCustomerCancelled + IsDriverCancelled) / COUNT(*) * 100, 2) as rate
        FROM uber_data_clean
        WHERE AvgVTAT > 0
        GROUP BY 
            CASE 
                WHEN AvgVTAT <= 2 THEN 'Under 2 min'
                WHEN AvgVTAT <= 5 THEN '2-5 min'
                WHEN AvgVTAT <= 10 THEN '5-10 min'
                WHEN AvgVTAT <= 15 THEN '10-15 min'
                ELSE 'Over 15 min'
            END
        ORDER BY rate DESC
        LIMIT 1
    ) as WaitBuckets
    ) as Value

Databricks visualization. Run in Databricks to view.